<a href="https://colab.research.google.com/github/rpine001/coursera_1/blob/master/Copy_of_getHistFMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

#https://www.the905guy.ca/article/how-to-create-a-relative-momentum-trading-strategy-in-python

def get_eod_data(symbol, session=None):
  if session is None:
    session = requests.Session()
    start = "2019-09-1"
    end = "2021-04-8"
    url = "https://eodhistoricaldata.com/api/eod/%s" % symbol
    params = {"api_token": "5c9ff10ca240a0.35985529","from":start,"to":end}
    r = session.get(url, params=params)
    if r.status_code == requests.codes.ok:
      #r_text = StringIO(r.text)
      #df = pd.read_csv(StringIO(r.text), skipfooter=1, parse_dates=[0], index_col=0, engine="python") #####DONT do this since can't use function more than once
      return StringIO(r.text) #r_text #df

  else:raise Exception(r.status_code, r.reason, url)


def momentum(closes):
    ##use linear regression of returns to measure risk adjust returns
    returns = np.log(closes)
    ##create index equal to length of array
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

def slopePrice(closes):
    ##use scipy.stats to calculate linear regression values
    y = closes
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return  intercept+slope*(t_lrch+lag_lrch)+std_err


#pd.set_option('display.max_rows', None)

#### Get list of assets



#my_df = pd.read_csv('mf_testing_data.csv')

#Get historic prices

#stocks = ['NVS.US']
#assetFile = "testlist1.csv"
#assetFile = "assets2.csv"
assetFile = "assets2.csv"
#assetFile ='/content/drive/MyDrive/Colab Data/assets2.csv'
stocks = pd.read_csv(assetFile, header=None)[0]
#stocks = ['NVS.US','AAPL.US','MSFT.US','GOOG.US','XLU.US','TLT.US','SPY.US']
#stocks = ['NVS.US','AAPL.US','SPY.US']
vols = ['VIX.INDX','VIX3M.INDX','VIX6M.INDX','VIX1Y.INDX']

rebal_freq=5
field = "Adjusted_close"
t_lrrsq = 60
t_lrch = 60
lag_lrch = 10

empresas = {}
empresas1 ={}
empresas2 ={}
empresasVIX={}
#Get all prices into a dataframe
for stock in stocks:
  #prices = get_eod_data(stock)
  prices = pd.read_csv(get_eod_data(stock), skipfooter=1, parse_dates=[0], index_col=0, engine="python")
  empresas1[stock] = prices[field].pct_change(1)
  #empresas[stock] = prices.set_index('Date') NOTE: not required since data is already indexed on Date
  #empresas[stock] = prices["Close"].pct_change(1).rolling(1).sum()/2+prices["Close"].pct_change().rolling(5).sum()/2
  empresas[stock] = prices[field].pct_change(5)/4+prices[field].pct_change(20)/2*0+prices[field].pct_change(60)/4*0
  #empresas[stock] = prices[field].rolling(t_lrrsq).apply(momentum, raw=False)
  empresas2[stock] = prices[field].rolling(t_lrch).apply(slopePrice, raw=False)


for v in vols:
  v_series = pd.read_csv(get_eod_data(v), skipfooter=1, parse_dates=[0], index_col=0, engine="python")
  empresasVIX[v] = v_series[field]


df_rtns = pd.concat(empresas1, axis=1) 

df_mmtm = pd.concat(empresas, axis=1)

df_lrch = pd.concat(empresas2, axis=1)



df_vix = pd.concat(empresasVIX, axis=1)
### Calc VIX filter: sig_VIX is when previous (VIX.INDX/VIX3M.INDX-1) > x%

df_vix['sigRtns'] = df_vix['VIX3M.INDX'].shift(periods=1, axis = 0)-df_vix['VIX.INDX'].shift(periods=1, axis = 0)
## add in calculation of zscore
#df_vix['zVIX'] = 
#df_vix['sig_VIX'] = np.where(df_vix['VIX.INDX'].shift(periods=1, axis = 0)/df_vix['VIX3M.INDX'].shift(periods=1, axis = 0)-1>-0.15, 1,1 )
df_vix['sig_VIX'] = np.where(df_vix['sigRtns']>-1, 1,0 )
df_rank = pd.DataFrame(columns=df_mmtm.columns, index=df_mmtm.index)


for i in range(len(df_rank)):
# calculate rank only on rebal date, NOTE: here % is the modulus function, i % rebal = 0 whenever i is a multiple of rebal_freq
  if i % rebal_freq == 0:
    df_rank.iloc[i] = df_mmtm.iloc[i].rank(axis=0, ascending=False)

#shift the rank observations to the next day - ie the day trade can occur
#then fill in the NaN between ranking dates with previous ranking

for i in range(len(df_vix)):
### calculate the lin regress slope of the vix futures
  x = np.arange(6)
  testval= linregress(x, df_vix.iloc[i]).slope
  #df_vix['slope'].iloc[i]= linregress(x, df_vix.iloc[i]).slope

df_rank = df_rank.shift(periods=1, axis = 0)
df_rank.ffill(inplace = True)
num_long = 5
num_short = 0
num_stocks = df_rank.shape[1] - num_short + 1
#allocate weight to top N and bottom N
for col in df_rank.columns:
    
    # if a stock has a rank in the top num_long values, we go long
 
    df_rank.loc[df_rank[col] <= num_long, col] = 1

    # if a stock has a rank in the bottom num_short values, we go short
    df_rank.loc[df_rank[col] >= num_stocks, col] = -0.5

    # if a stock doesn't have a ranking in top or bottom n values, we have no position
    df_rank.loc[(df_rank[col] < num_stocks) & (df_rank[col] > num_long), col] = None

#get daily changes from original prices file and multiply by rank wgts
df_wgtRtn = df_rtns*df_rank
#divide wgt rtn by the number of stock bought and sold
df_wgtRtn["Row Rtn"] = df_wgtRtn.sum(axis=1, skipna=True)/(num_long+num_short) 
df_wgtRtn["rtnBM"] =df_mmtm['SPY.US']
df_wgtRtn['sigSPY'] = 1
df_wgtRtn['Exit'] = np.where(df_mmtm['SPY.US'].shift(periods=1, axis = 0)>0, 1,1 )
df_wgtRtn=pd.concat([df_wgtRtn, df_vix[['sig_VIX']]], axis=1, sort=False, join='inner')
df_wgtRtn=pd.concat([df_wgtRtn, df_vix[['sigRtns']]], axis=1, sort=False, join='inner')
df_wgtRtn["Portfolio Value"] = (df_wgtRtn["Row Rtn"]*df_wgtRtn['Exit']*df_wgtRtn['sig_VIX']+1).cumprod()*1
df_wgtRtn["BMrk"] =(df_rtns["SPY.US"] +1).cumprod()*1



#df_wgtRtn['Exit']
#df_wgtRtn["rtnBM"]
df_wgtRtn['Portfolio Value'].plot()
df_wgtRtn['BMrk'].plot()


df_test = df_vix['VIX.INDX'].shift(periods=1, axis = 0)/df_vix['VIX3M.INDX'].shift(periods=1, axis = 0)-1
#print (df_test)
df_test.plot()
#print (df_vix)

plt.title('Equity Curve for Relative Momentum Strategy')
plt.show()
plt.title('Scatter')
plt.plot(df_wgtRtn['sigRtns'],df_wgtRtn["SPY.US"], 'o' )

df_rank.to_csv ('/content/drive/MyDrive/Colab Data/rank.csv', index = True, header=True)

print(df_lrch)  
#return_stocks = portfolio.pct_change()
#df_wgtRtn
#bests
#df_rtns




FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

# New Section